<span style="font-size:30px">DATA PREPROSSESERING</span>

Biblotek:

In [33]:
import os
import numpy as np 
import time
import torch
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader 
from chess import pgn 
from tqdm import tqdm 

Last inn data fra spillene

In [36]:
# LOAD DATA

def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)          # lagrer spillene som et objekt av typen Game fra chess bibloteket
            if game is None:
                break
            games.append(game)
    return games

files = [file for file in os.listdir("game data/Lichess Elite Database") if file.endswith(".pgn")]
LIMIT_OF_FILES = min(len(files), 2)
games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"game data\Lichess Elite Database\{file}"))
    if i >= LIMIT_OF_FILES:
        break
    i += 1

print(f"Spill parsed: {len(games)}")


  0%|          | 0/7 [01:32<?, ?it/s]


KeyboardInterrupt: 

KONVERTER DATA TIL TENSORS

In [35]:
from auxillary_func import create_input_for_nn, encode_moves

# tar 6 min å kjøre :(
X, y = create_input_for_nn(games)   # hvert element av X er hver stilling i løpet av spillet og y er et trekk

print(f"NUMBER OF SAMPLES: {len(y)}")


MemoryError: Unable to allocate 6.50 KiB for an array with shape (13, 8, 8) and data type float64

In [ ]:
# gjør det litt mindre
X = X[0:2500000]        # X er tilstanden til brettet
y = y[0:2500000]        # y er på formen: ['e2e4' 'd7d5' 'e4d5' 'd8d5' 'b1c3' 'd5a5' 'd2d4' 'c7c6' 'g1f3' 'g8f6']

print(y[:10])

['e2e4' 'd7d5' 'e4d5' 'd8d5' 'b1c3' 'd5a5' 'd2d4' 'c7c6' 'g1f3' 'g8f6']


In [ ]:
y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)   

<span style="font-size:40px">Oppsett for PyTorch</span>
1. lag klasse for datasett
2. lag klasse for modellen
3. lag treningsoppsett

In [ ]:
from dataset import ChessDataset
from model import ChessModel

In [ ]:
# opprett datasett og dataloader
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Sjekker GPU greier
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Oppretter modellen
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cpu


<span style="font-size:40px">Trening av modellen</span>

In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0

    # iterer over datasettet
    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)  # flytt data til GPU
        optimizer.zero_grad()

        outputs = model(inputs)  

        # beregn loss
        loss = criterion(outputs, labels)
        loss.backward()
        
        # gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item()
    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time) - minutes * 60
    print(f'Epoch {epoch + 1 + 50}/{num_epochs + 1 + 50}, Loss: {running_loss / len(dataloader):.4f}, Time: {minutes}m{seconds}s')

  0%|          | 0/39063 [00:00<?, ?it/s]

  5%|▍         | 1863/39063 [01:10<23:25, 26.47it/s]


KeyboardInterrupt: 

Lagre modellen
Trene modellen tar timesvis så vil helst bare gjøre det en gang

In [ ]:
# lagre modellen
torch.save(model.state_dict(), "models/TORCH_100EPOCHS.pth")

import pickle

with open("heavy_move_to_int", "wb") as file:
    pickle.dump(move_to_int, file)